In [6]:
# 1. 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 2. 압축 파일 복사 (드라이브 -> VM 로컬)
!cp /content/drive/MyDrive/final_dataset.zip /content/

# 3. 압축 해제
!unzip -q /content/final_dataset.zip -d /content/dataset

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import os
import numpy as np
import librosa
from scipy.signal import butter, lfilter

# 버터워스 대역 통과 필터 설계
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def apply_filter(data, lowcut=500, highcut=2000, fs=16000):
    b, a = butter_bandpass(lowcut, highcut, fs)
    return lfilter(b, a, data)

def preprocess_all(base_path):
    classes = ['class_0_noise', 'class_1_sneeze']
    X_data = []
    y_data = []
    sr = 16000
    target_len = int(sr * 2.0)

    for idx, cls in enumerate(classes):
        path = os.path.join(base_path, cls)
        files = [f for f in os.listdir(path) if f.endswith('.wav')]
        print(f'{cls} 처리 중: {len(files)}개')

        for f in files:
            audio, _ = librosa.load(os.path.join(path, f), sr=sr)
            audio = librosa.util.fix_length(audio, size=target_len)
            
            # 필터 적용 및 에너지 정규화
            filtered = apply_filter(audio)
            rms = np.sqrt(np.mean(filtered**2))
            if rms > 0:
                filtered = filtered / rms * 0.1
                
            mfcc = librosa.feature.mfcc(y=filtered, sr=sr, n_mfcc=20, hop_length=512)
            X_data.append(mfcc)
            y_data.append(idx)

    np.save('X_filtered.npy', np.array(X_data))
    np.save('y_filtered.npy', np.array(y_data))
    print('X_filtered.npy 및 y_filtered.npy 저장 완료')

In [13]:
preprocess_all('dataset')

class_0_noise 처리 중: 15120개
class_1_sneeze 처리 중: 10080개
X_filtered.npy 및 y_filtered.npy 저장 완료


In [14]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

# 데이터 로드
X = np.load('X_filtered.npy')
y = np.load('y_filtered.npy')

# 차원 조정 (batch, time, freq, channel)
X = X.transpose(0, 2, 1)
X = X[..., np.newaxis]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 스케일러 적용 및 저장
scaler = StandardScaler()
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
X_test_reshaped = X_test.reshape(X_test.shape[0], -1)

scaler.fit(X_train_reshaped)
joblib.dump(scaler, 'sneeze_scaler_filtered.pkl')

X_train_scaled = scaler.transform(X_train_reshaped).reshape(X_train.shape)
X_test_scaled = scaler.transform(X_test_reshaped).reshape(X_test.shape)

# DS-CRNN 모델 정의
def build_ds_crnn(input_shape):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.DepthwiseConv2D(kernel_size=(3, 3), padding='same', activation='relu'),
        layers.Conv2D(32, (1, 1), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Reshape((input_shape[0]//2, -1)),
        layers.GRU(64, return_sequences=False),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

model = build_ds_crnn(X_train_scaled.shape[1:])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 학습
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, 
          validation_split=0.2, callbacks=[early_stop])

model.save('sneeze_model_filtered.keras')

Epoch 1/100
504/504 ━━━━━━━━━━━━━━━━━━━━ 37s 67ms/step - accuracy: 0.6849 - loss: 0.5837 - val_accuracy: 0.7736 - val_loss: 0.4757
Epoch 2/100
504/504 ━━━━━━━━━━━━━━━━━━━━ 33s 66ms/step - accuracy: 0.7752 - loss: 0.4832 - val_accuracy: 0.7815 - val_loss: 0.4643
Epoch 3/100
504/504 ━━━━━━━━━━━━━━━━━━━━ 34s 68ms/step - accuracy: 0.8197 - loss: 0.4155 - val_accuracy: 0.8524 - val_loss: 0.3272
Epoch 4/100
504/504 ━━━━━━━━━━━━━━━━━━━━ 42s 70ms/step - accuracy: 0.8578 - loss: 0.3408 - val_accuracy: 0.8795 - val_loss: 0.2845
Epoch 5/100
504/504 ━━━━━━━━━━━━━━━━━━━━ 33s 65ms/step - accuracy: 0.8898 - loss: 0.2814 - val_accuracy: 0.8993 - val_loss: 0.2467
Epoch 6/100
504/504 ━━━━━━━━━━━━━━━━━━━━ 41s 65ms/step - accuracy: 0.8991 - loss: 0.2504 - val_accuracy: 0.8996 - val_loss: 0.2459
Epoch 7/100
504/504 ━━━━━━━━━━━━━━━━━━━━ 34s 68ms/step - accuracy: 0.9201 - loss: 0.2135 - val_accuracy: 0.9125 - val_loss: 0.2103
Epoch 8/100
504/504 ━━━━━━━━━━━━━━━━━━━━ 42s 70ms/step - accuracy: 0.9268 - loss: 0

In [17]:
!cp sneeze_model_filtered.keras /content/drive/MyDrive/

In [19]:
!cp sneeze_scaler_filtered.pkl /content/drive/MyDrive/